## Import and Data Load

In [39]:
import pandas as pd
import numpy as np
import json
import requests
import time

In [40]:
!pip install unidecode

## BERT - Pretrained models

In [41]:
#dataset = pd.read_excel("Dataset Pmc T2T.xlsx")
dataset = pd.read_excel("Dataset Nature T2T.xlsx")
dataset = dataset[dataset.Abstract != ""]
dataset

,PMID,Title,Abstract,PubTator_simplified,T5_Q1,ALPACA_Q1,LAMINI_Q1,Unnamed: 7,Unnamed: 8,T5_Q2,ALPACA_Q2,LAMINI_Q2,Unnamed: 12,Unnamed: 13,T5_Q3,ALPACA_Q3,LAMINI_Q3,Unnamed: 17,Unnamed: 18
0,31358913.0,2-Pyrazol-1-yl-thiazole derivatives as novel h...,The present report describes our efforts to id...,"['2-pyrazol-1-yl-thiazole', '2-pyrazol-1-yl-th...",<pad> yes</s>,"Yes, context: This report describes our effort...",Yes.,3,YES,<pad> yes</s>,Yes,No,2,YES,<pad> yes</s>,Yes,Yes.,3,YES
1,30127421.0,A choline binding polypeptide of LytA inhibits...,Streptococcus pneumoniae is a pathogen that ma...,['choline'],<pad> yes</s>,"No, context: Streptococcus pneumoniae is a pat...",Yes.,2,YES,<pad> yes</s>,Yes,Yes.,3,YES,<pad> yes</s>,"Yes, simulated choline binding polypeptide (Ch...",Yes.,3,YES
2,34987225.0,A naturally inspired antibiotic to target mult...,Gram-negative bacteria are responsible for an ...,"['macolacin', 'biphenyl', 'colistin3']",<pad> yes</s>,"Yes, context: Gram-negative bacteria are respo...",Yes.,3,YES,<pad> yes</s>,Yes,Yes.,3,YES,<pad> yes</s>,"No, context: Gram-negative bacteria are respon...",Yes.,2,YES
3,29590091.0,A new class of synthetic retinoid antibiotics ...,A challenge in the treatment of Staphylococcus...,"['methicillin', 'CD437', 'retinoids', 'gentami...",<pad> yes</s>,Yes,Yes.,3,YES,<pad> yes</s>,"Yes, there is a reference to the use of chemic...",Yes.,3,YES,<pad> yes</s>,Yes,Yes.,3,YES
4,32499514.0,"A novel, rationally designed, hybrid antimicro...",Antimicrobial peptides (AMPs) are promising al...,"['AMPs', 'Antimicrobial peptides', 'LPS', 'lip...",<pad> yes</s>,"Yes, context: Antimicrobial peptides (AMPs) ar...",Yes.,3,YES,<pad> yes</s>,"Yes, there is a reference to the use of chemic...",Yes.,3,YES,<pad> yes</s>,"No, this study revealed the therapeutic potenc...",Yes.,2,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,36071214.0,"Wychimicins, a new class of spirotetronate pol...",In the course of our screening program for new...,"['methicillin', 'beta-d-xylo-hexopyranose', 'W...",<pad> yes</s>,"No, context: In the course of our screening pr...",Yes.,2,YES,<pad> yes</s>,"Yes, there is a reference to the use of chemic...",Yes.,3,YES,<pad> yes</s>,"No, the wychimicins A-D antibiotics have a min...",Yes.,2,YES
79,33741965.0,trans-Translation inhibitors bind to a novel s...,Bacterial ribosome rescue pathways that remove...,"['MBX-4132', 'acylaminooxadiazoles']",<pad> yes</s>,"No, context: Bacterial ribosome rescue pathway...",Yes.,2,YES,<pad> yes</s>,"Yes, there is a reference to the use of chemic...",Yes.,3,YES,<pad> yes</s>,"No, context: Bacterial ribosome rescue pathway...",Yes.,2,YES
80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,77,NaN,NaN,NaN,YES,80,NaN,NaN,NaN,YES,76


In [42]:
dataset.dropna(subset="Abstract", inplace=True)

In [43]:
titles = dataset['Title'].tolist()
abstracts = dataset['Abstract'].tolist()

In [44]:
id_list = dataset['PMID'].values
#real_list = dataset['Target'].values

In [45]:
from unidecode import unidecode

def text_process(text, type_='list'):
    preproc = text.lower()
    preproc = unidecode(preproc)
    if type_=='list':
        preproc = preproc.split(',')
    return preproc

abstracts = [text_process(text, type_='notlist') for text in dataset.Abstract.tolist()]
dataset['Abstract_preproc'] = abstracts

In [46]:
import re

def compare_rows(text, textlist):
    compare = False
    if text in textlist or text+'s' in textlist:
        compare = True
    elif '(' in text:
        newt = re.sub(r'\([^)]*\)', '', text)
        if newt in textlist or newt+'s' in textlist:
            compare = True
    return compare

In [47]:
from transformers import pipeline
from transformers import AutoTokenizer

base_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def hugging_model_process(model_name, tokenizer, text, offset=0):

    token = model_name if tokenizer else base_tokenizer
    ner_model = pipeline(task="ner", model=model_name, tokenizer=token)
    model_res = ner_model(text, aggregation_strategy="first")

    array_res = []
    for dic in model_res:
        annot_type = dic['entity_group']
        if annot_type.lower() in ['chemical','bio','b']:
            annot_text = dic['word']
            annot_loc = {'start': dic['start'] + offset, 'end': dic['end'] + offset}
            res = {'text': annot_text, 'location': annot_loc}
            array_res.append(res)
    return array_res

In [48]:
hf_models = ["raynardj/ner-chemical-bionlp-bc5cdr-pubmed",
             "alvaroalon2/biobert_chemical_ner",
             "aitslab/biobert_huner_chemical_v1",
             "pruas/BENT-PubMedBERT-NER-Chemical"]

#### Raynard

In [49]:
raylist = []
for i, text in enumerate(abstracts):
    offset = len(titles[i])
    raychemres = hugging_model_process(hf_models[0], True, text, offset)
    rayres = {'id': id_list[i], 'chemical': raychemres}
    raylist.append(rayres)

Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/token_classification.py:490: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu


In [50]:
ray_list = []

for id in raylist:
    chemlist = []
    for chem in id['chemical']:
        chemlist.append(chem['text'].lower()[1:])
    ray_list.append(chemlist)

print(len(ray_list))

80


In [51]:
len(ray_list)

80

In [52]:
dataset['Raynardj'] = ray_list

#### AlonBioBert

In [53]:
alonlist = []
for i, text in enumerate(abstracts):
    offset = len(titles[i])
    alonchemres = hugging_model_process(hf_models[1], True, text, offset)
    alonres = {'id': id_list[i], 'chemical': alonchemres}
    alonlist.append(alonres)

Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set

In [54]:
alon_list = []

for id in alonlist:
    chemlist = []
    for chem in id['chemical']:
        chemlist.append(chem['text'].lower())
    alon_list.append(chemlist)

print(len(alon_list))

80


In [55]:
len(alon_list)

80

In [56]:
dataset['AlvaroAlon'] = alon_list

#### AitsLab

In [57]:
aitslist = []
for i, text in enumerate(abstracts):
    offset = len(titles[i])
    aitschemres = hugging_model_process(hf_models[2], False, text, offset)
    aitsres = {'id': id_list[i], 'chemical': aitschemres}
    aitslist.append(aitsres)

pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

Device set to use cpu


model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set

In [58]:
aits_list = []

for id in aitslist:
    chemlist = []
    for chem in id['chemical']:
        chemlist.append(chem['text'].lower())
    aits_list.append(chemlist)

print(len(aits_list))

80


In [59]:
dataset['AitsLab'] = aits_list

#### Pruas

In [60]:
ner_model = pipeline(task="ner", model=hf_models[3], tokenizer=base_tokenizer)
model_res = ner_model(abstracts[0], aggregation_strategy="first")
print(model_res)

config.json:   0%|          | 0.00/803 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Device set to use cpu


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

[]


In [61]:
pruaslist = []
for i, text in enumerate(abstracts):
    offset = len(titles[i])
    try:
        pruaschemres = hugging_model_process(hf_models[3], True, text, offset)
        pruasres = {'id': id_list[i], 'chemical': pruaschemres}
    except:
        pruasres = {'id': id_list[i], 'chemical': ''}
    pruaslist.append(pruasres)

tokenizer_config.json:   0%|          | 0.00/379 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Device set to use cpu


In [62]:
pruaslist[0]

{'id': np.float64(31358913.0),
 'chemical': [{'text': '2', 'location': {'start': 582, 'end': 583}},
  {'text': 'erythromycin', 'location': {'start': 942, 'end': 954}},
  {'text': 'levofloxacin', 'location': {'start': 1027, 'end': 1039}},
  {'text': '2', 'location': {'start': 1295, 'end': 1296}}]}

In [63]:
pruas_list = []

for id in pruaslist:
    chemlist = []
    for chem in id['chemical']:
        chemlist.append(chem['text'].lower())
    pruas_list.append(chemlist)

print(len(pruas_list))

80


In [64]:
dataset['Pruas'] = pruas_list

## Export

In [65]:
reduce = lambda l: np.unique(l)

dataset['Raynardj'] = dataset['Raynardj'].apply(reduce)
dataset['AlvaroAlon'] = dataset['AlvaroAlon'].apply(reduce)
dataset['AitsLab'] = dataset['AitsLab'].apply(reduce)
dataset['Pruas'] = dataset['Pruas'].apply(reduce)

In [66]:
dataset

,PMID,Title,Abstract,PubTator_simplified,T5_Q1,ALPACA_Q1,LAMINI_Q1,Unnamed: 7,Unnamed: 8,T5_Q2,...,T5_Q3,ALPACA_Q3,LAMINI_Q3,Unnamed: 17,Unnamed: 18,Abstract_preproc,Raynardj,AlvaroAlon,AitsLab,Pruas
0,31358913.0,2-Pyrazol-1-yl-thiazole derivatives as novel h...,The present report describes our efforts to id...,"['2-pyrazol-1-yl-thiazole', '2-pyrazol-1-yl-th...",<pad> yes</s>,"Yes, context: This report describes our effort...",Yes.,3,YES,<pad> yes</s>,...,<pad> yes</s>,Yes,Yes.,3,YES,the present report describes our efforts to id...,"[2-pyrazol-1-yl-thiazole, 2-pyrazol-1-yl-thiaz...","[2 - pyrazol - 1 - yl - thiazole, 2 - pyrazol ...","[2 - pyrazol - 1 - yl - thiazole, 2 - pyrazol ...","[2, erythromycin, levofloxacin]"
1,30127421.0,A choline binding polypeptide of LytA inhibits...,Streptococcus pneumoniae is a pathogen that ma...,['choline'],<pad> yes</s>,"No, context: Streptococcus pneumoniae is a pat...",Yes.,2,YES,<pad> yes</s>,...,<pad> yes</s>,"Yes, simulated choline binding polypeptide (Ch...",Yes.,3,YES,streptococcus pneumoniae is a pathogen that ma...,[choline],"[chbp, choline]",[choline],[choline]
2,34987225.0,A naturally inspired antibiotic to target mult...,Gram-negative bacteria are responsible for an ...,"['macolacin', 'biphenyl', 'colistin3']",<pad> yes</s>,"Yes, context: Gram-negative bacteria are respo...",Yes.,3,YES,<pad> yes</s>,...,<pad> yes</s>,"No, context: Gram-negative bacteria are respon...",Yes.,2,YES,gram-negative bacteria are responsible for an ...,"[biphenyl, colistin, colistin-resistance, coli...","[biphenyl, colistin, colistin3, macolacin, pho...","[biphenyl, colistin, colistin3, macolacin, pho...","[biphenyl, colistin, colistin3, macolacin, pho..."
3,29590091.0,A new class of synthetic retinoid antibiotics ...,A challenge in the treatment of Staphylococcus...,"['methicillin', 'CD437', 'retinoids', 'gentami...",<pad> yes</s>,Yes,Yes.,3,YES,<pad> yes</s>,...,<pad> yes</s>,Yes,Yes.,3,YES,a challenge in the treatment of staphylococcus...,"[cd1530, cd1530,, cd437, gentamicin,, methicil...","[cd1530, cd437, gentamicin, methicillin, retin...","[cd1530, cd437, gentamicin, methicillin, retin...","[gentamicin, methicillin, retinoids]"
4,32499514.0,"A novel, rationally designed, hybrid antimicro...",Antimicrobial peptides (AMPs) are promising al...,"['AMPs', 'Antimicrobial peptides', 'LPS', 'lip...",<pad> yes</s>,"Yes, context: Antimicrobial peptides (AMPs) ar...",Yes.,3,YES,<pad> yes</s>,...,<pad> yes</s>,"No, this study revealed the therapeutic potenc...",Yes.,2,YES,antimicrobial peptides (amps) are promising al...,[lipopolysaccharide],"[amino acid, c]","[amino acid, aurein, cathelicidin]","[amino, c]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,30181560.0,Total synthesis and antimicrobial evaluation o...,Development of effective antimicrobial agents ...,"['albomycins', 'albomycins delta1 (1a), delta2...",<pad> yes</s>,"No, context: Development of effective antimicr...",Yes.,2,YES,<pad> yes</s>,...,<pad> yes</s>,"No, context: Development of effective antimicr...",Yes.,2,YES,development of effective antimicrobial agents ...,"[albomycin, albomycins]","[), i ' 2 ( 1b ), and iu, albomycin, albomycin...","[albomycin, albomycin i ' 2, albomycins, albom...","[albomycin, albomycins]"
76,32424122.0,"Venturicidin A, A Membrane-active Natural Prod...",Despite the remarkable advances due to the dis...,"['methicillin', 'VentA', 'aminoglycoside', 'ge...",<pad> yes</s>,Yes,Yes.,3,YES,<pad> yes</s>,...,<pad> yes</s>,"No, antibiotics are not used against biofilm, ...",Yes.,2,YES,despite the remarkable advances due to the dis...,"[aminoglycoside, gentamicin, methicillin-resis...","[aminoglycoside, atp, gentamicin, methicillin,...","[aminoglycoside, atp, gentamicin, methicillin,...","[aminoglycoside, atp, gentamicin, methicillin,..."
77,20057515.0,"Walkmycin B targets WalK (YycG), a histidine k...",The WalK (a histidine kinase)/WalR (a response...,"['silica', 'C(44)H(44)Cl(2)O(

In [67]:
def process_response(response):
    M = []
    if len(response) == 0:
        return np.array([])

    for elem in response:
        transf = re.sub(r",|\.",'',elem)
        transf = re.sub(r"'",'',transf)
        transf = re.sub(r"\s\-\s",'-',transf)
        transf = re.sub(r"\s?\(\s?",'(',transf)
        transf = re.sub(r"\s?\)\s?",')',transf)
        transf = re.sub(r"\s{2,}",' ',transf)
        M.append(transf)
    return np.unique(M)

In [68]:
dataset['Raynardj'] = dataset['Raynardj'].apply(process_response)
dataset['AlvaroAlon'] = dataset['AlvaroAlon'].apply(process_response)
dataset['AitsLab'] = dataset['AitsLab'].apply(process_response)
dataset['Pruas'] = dataset['Pruas'].apply(process_response)

In [69]:
all_res = [ np.concatenate((dataset['Raynardj'][i].astype(list),
                          dataset['AlvaroAlon'][i].astype(list),
                          dataset['AitsLab'][i].astype(list),
                          dataset['Pruas'][i].astype(list))) for i in list(dataset.index)]

one_res = [np.unique(res) for res in all_res]
dataset['Combined words 1/4'] = one_res

In [70]:
dataset

,PMID,Title,Abstract,PubTator_simplified,T5_Q1,ALPACA_Q1,LAMINI_Q1,Unnamed: 7,Unnamed: 8,T5_Q2,...,ALPACA_Q3,LAMINI_Q3,Unnamed: 17,Unnamed: 18,Abstract_preproc,Raynardj,AlvaroAlon,AitsLab,Pruas,Combined words 1/4
0,31358913.0,2-Pyrazol-1-yl-thiazole derivatives as novel h...,The present report describes our efforts to id...,"['2-pyrazol-1-yl-thiazole', '2-pyrazol-1-yl-th...",<pad> yes</s>,"Yes, context: This report describes our effort...",Yes.,3,YES,<pad> yes</s>,...,Yes,Yes.,3,YES,the present report describes our efforts to id...,"[2-pyrazol-1-yl-thiazole, 2-pyrazol-1-yl-thiaz...","[2-pyrazol-1-yl-thiazole, 2-pyrazol-1-yl-thiaz...","[2-pyrazol-1-yl-thiazole, 2-pyrazol-1-yl-thiaz...","[2, erythromycin, levofloxacin]","[2, 2-pyrazol-1-yl-thiazole, 2-pyrazol-1-yl-th..."
1,30127421.0,A choline binding polypeptide of LytA inhibits...,Streptococcus pneumoniae is a pathogen that ma...,['choline'],<pad> yes</s>,"No, context: Streptococcus pneumoniae is a pat...",Yes.,2,YES,<pad> yes</s>,...,"Yes, simulated choline binding polypeptide (Ch...",Yes.,3,YES,streptococcus pneumoniae is a pathogen that ma...,[choline],"[chbp, choline]",[choline],[choline],"[chbp, choline]"
2,34987225.0,A naturally inspired antibiotic to target mult...,Gram-negative bacteria are responsible for an ...,"['macolacin', 'biphenyl', 'colistin3']",<pad> yes</s>,"Yes, context: Gram-negative bacteria are respo...",Yes.,3,YES,<pad> yes</s>,...,"No, context: Gram-negative bacteria are respon...",Yes.,2,YES,gram-negative bacteria are responsible for an ...,"[biphenyl, colistin, colistin-resistance, coli...","[biphenyl, colistin, colistin3, macolacin, pho...","[biphenyl, colistin, colistin3, macolacin, pho...","[biphenyl, colistin, colistin3, macolacin, pho...","[biphenyl, colistin, colistin-resistance, coli..."
3,29590091.0,A new class of synthetic retinoid antibiotics ...,A challenge in the treatment of Staphylococcus...,"['methicillin', 'CD437', 'retinoids', 'gentami...",<pad> yes</s>,Yes,Yes.,3,YES,<pad> yes</s>,...,Yes,Yes.,3,YES,a challenge in the treatment of staphylococcus...,"[cd1530, cd437, gentamicin, methicillin-resist...","[cd1530, cd437, gentamicin, methicillin, retin...","[cd1530, cd437, gentamicin, methicillin, retin...","[gentamicin, methicillin, retinoids]","[cd1530, cd437, gentamicin, methicillin, methi..."
4,32499514.0,"A novel, rationally designed, hybrid antimicro...",Antimicrobial peptides (AMPs) are promising al...,"['AMPs', 'Antimicrobial peptides', 'LPS', 'lip...",<pad> yes</s>,"Yes, context: Antimicrobial peptides (AMPs) ar...",Yes.,3,YES,<pad> yes</s>,...,"No, this study revealed the therapeutic potenc...",Yes.,2,YES,antimicrobial peptides (amps) are promising al...,[lipopolysaccharide],"[amino acid, c]","[amino acid, aurein, cathelicidin]","[amino, c]","[amino, amino acid, aurein, c, cathelicidin, l..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,30181560.0,Total synthesis and antimicrobial evaluation o...,Development of effective antimicrobial agents ...,"['albomycins', 'albomycins delta1 (1a), delta2...",<pad> yes</s>,"No, context: Development of effective antimicr...",Yes.,2,YES,<pad> yes</s>,...,"No, context: Development of effective antimicr...",Yes.,2,YES,development of effective antimicrobial agents ...,"[albomycin, albomycins]","[)i 2(1b)and iu, albomycin, albomycin i 2, alb...","[albomycin, albomycin i 2, albomycins, albomyc...","[albomycin, albomycins]","[)i 2(1b)and iu, albomycin, albomycin i 2, alb..."
76,32424122.0,"Venturicidin A, A Membrane-active Natural Prod...",Despite the remarkable advances due to the dis...,"['methicillin', 'VentA', 'aminoglycoside', 'ge...",<pad> yes</s>,Yes,Yes.,3,YES,<pad> yes</s>,...,"No, antibiotics are not used against biofilm, ...",Yes.,2,YES,despite the remarkable advances due to the dis...,"[aminoglycoside, gentamicin, methicillin-resis...","[aminoglycoside, atp, gentamicin, methicillin,...","[aminoglycoside, atp, gentamicin, methicillin,...","[ami

In [71]:
all_res = [np.unique(res, return_counts=True) for res in all_res]
all_res

[(array(['2', '2-pyrazol-1-yl-thiazole', '2-pyrazol-1-yl-thiazoles',
         'erythromycin', 'levofloxacin'], dtype=object),
  array([1, 3, 3, 4, 4])),
 (array(['chbp', 'choline'], dtype=object), array([1, 4])),
 (array(['biphenyl', 'colistin', 'colistin-resistance',
         'colistin-resistant', 'colistin3', 'macolacin',
         'phosphoethanolamine'], dtype=object),
  array([4, 4, 1, 1, 4, 4, 3])),
 (array(['cd1530', 'cd437', 'gentamicin', 'methicillin',
         'methicillin-resistant', 'retinoids', 'retinoids cd437'],
        dtype=object),
  array([3, 3, 4, 3, 1, 4, 1])),
 (array(['amino', 'amino acid', 'aurein', 'c', 'cathelicidin',
         'lipopolysaccharide'], dtype=object),
  array([1, 2, 1, 2, 1, 1])),
 (array(['azetidine', 'azetidine derivative brd4592', 'brd4592', 'i', 'i +',
         'tryptophan'], dtype=object),
  array([3, 1, 4, 1, 1, 3])),
 (array(['aminooctose', 'clindamycin', 'iboxamycin',
         'iboxamycin iboxamycin', 'lincosamides', 'macrolides',
         '

In [72]:
dataset['Combined words 2/4'] = [all_res[i][0][all_res[i][1] >= 2] for i in range(len(all_res))]
dataset['Combined words 3/4'] = [all_res[i][0][all_res[i][1] >= 3] for i in range(len(all_res))]
dataset['Combined words 4/4'] = [all_res[i][0][all_res[i][1] == 4] for i in range(len(all_res))]

In [73]:
dataset

,PMID,Title,Abstract,PubTator_simplified,T5_Q1,ALPACA_Q1,LAMINI_Q1,Unnamed: 7,Unnamed: 8,T5_Q2,...,Unnamed: 18,Abstract_preproc,Raynardj,AlvaroAlon,AitsLab,Pruas,Combined words 1/4,Combined words 2/4,Combined words 3/4,Combined words 4/4
0,31358913.0,2-Pyrazol-1-yl-thiazole derivatives as novel h...,The present report describes our efforts to id...,"['2-pyrazol-1-yl-thiazole', '2-pyrazol-1-yl-th...",<pad> yes</s>,"Yes, context: This report describes our effort...",Yes.,3,YES,<pad> yes</s>,...,YES,the present report describes our efforts to id...,"[2-pyrazol-1-yl-thiazole, 2-pyrazol-1-yl-thiaz...","[2-pyrazol-1-yl-thiazole, 2-pyrazol-1-yl-thiaz...","[2-pyrazol-1-yl-thiazole, 2-pyrazol-1-yl-thiaz...","[2, erythromycin, levofloxacin]","[2, 2-pyrazol-1-yl-thiazole, 2-pyrazol-1-yl-th...","[2-pyrazol-1-yl-thiazole, 2-pyrazol-1-yl-thiaz...","[2-pyrazol-1-yl-thiazole, 2-pyrazol-1-yl-thiaz...","[erythromycin, levofloxacin]"
1,30127421.0,A choline binding polypeptide of LytA inhibits...,Streptococcus pneumoniae is a pathogen that ma...,['choline'],<pad> yes</s>,"No, context: Streptococcus pneumoniae is a pat...",Yes.,2,YES,<pad> yes</s>,...,YES,streptococcus pneumoniae is a pathogen that ma...,[choline],"[chbp, choline]",[choline],[choline],"[chbp, choline]",[choline],[choline],[choline]
2,34987225.0,A naturally inspired antibiotic to target mult...,Gram-negative bacteria are responsible for an ...,"['macolacin', 'biphenyl', 'colistin3']",<pad> yes</s>,"Yes, context: Gram-negative bacteria are respo...",Yes.,3,YES,<pad> yes</s>,...,YES,gram-negative bacteria are responsible for an ...,"[biphenyl, colistin, colistin-resistance, coli...","[biphenyl, colistin, colistin3, macolacin, pho...","[biphenyl, colistin, colistin3, macolacin, pho...","[biphenyl, colistin, colistin3, macolacin, pho...","[biphenyl, colistin, colistin-resistance, coli...","[biphenyl, colistin, colistin3, macolacin, pho...","[biphenyl, colistin, colistin3, macolacin, pho...","[biphenyl, colistin, colistin3, macolacin]"
3,29590091.0,A new class of synthetic retinoid antibiotics ...,A challenge in the treatment of Staphylococcus...,"['methicillin', 'CD437', 'retinoids', 'gentami...",<pad> yes</s>,Yes,Yes.,3,YES,<pad> yes</s>,...,YES,a challenge in the treatment of staphylococcus...,"[cd1530, cd437, gentamicin, methicillin-resist...","[cd1530, cd437, gentamicin, methicillin, retin...","[cd1530, cd437, gentamicin, methicillin, retin...","[gentamicin, methicillin, retinoids]","[cd1530, cd437, gentamicin, methicillin, methi...","[cd1530, cd437, gentamicin, methicillin, retin...","[cd1530, cd437, gentamicin, methicillin, retin...","[gentamicin, retinoids]"
4,32499514.0,"A novel, rationally designed, hybrid antimicro...",Antimicrobial peptides (AMPs) are promising al...,"['AMPs', 'Antimicrobial peptides', 'LPS', 'lip...",<pad> yes</s>,"Yes, context: Antimicrobial peptides (AMPs) ar...",Yes.,3,YES,<pad> yes</s>,...,YES,antimicrobial peptides (amps) are promising al...,[lipopolysaccharide],"[amino acid, c]","[amino acid, aurein, cathelicidin]","[amino, c]","[amino, amino acid, aurein, c, cathelicidin, l...","[amino acid, c]",[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,30181560.0,Total synthesis and antimicrobial evaluation o...,Development of effective antimicrobial agents ...,"['albomycins', 'albomycins delta1 (1a), delta2...",<pad> yes</s>,"No, context: Development of effective antimicr...",Yes.,2,YES,<pad> yes</s>,...,YES,development of effective antimicrobial agents ...,"[albomycin, albomycins]","[)i 2(1b)and iu, albomycin, albomycin i 2, alb...","[albomycin, albomycin i 2, albomycins, albomyc...","[albomycin, albomycins]","[)i 2(1b)and iu, albomycin, albomycin i 2, alb...","[albomycin, albomycin i 2, albomycins, albomyc...","[albomycin, albomycins]","[albomycin, albomycins]"
76,32424122.0,"Venturicidin A, A Membrane-active Natural Prod...",Despite the remarkable advances due to the dis...,"['methicillin', 'VentA', 'aminogl

In [74]:
#dataset.to_excel('DatasetPositivoNER1.xlsx', index=False)
dataset.to_excel('DatasetPositivoNER2.xlsx', index=False)

## THESE TWO FILES ARE JOINED TOGETHER TO MAKE DatasetPositiveAll.xlsx